In [1]:
import math
import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np
import xarray as xr
import pickle
import netCDF4 as nc

sys.path.append('D:\\mehdi\\WaterBalance_new\\')
from qdwb.evapotranspiration.et import *
from qdwb.evapotranspiration.asset import *
from qdwb.evapotranspiration.convert import *
from qdwb.primary_surface_flow.primary_surface_flow import *
from qdwb.primary_surface_flow.asset import *
from qdwb.primary_surface_flow.check import *
from qdwb.soil_content.soil_layers import *


In [2]:
data_path = "D:\\mehdi\\DailyData\\Data\\"

# list of name of files in folder
def list_of_files(data_path):    
    files = os.listdir(data_path)
    return files

all_list_files = list_of_files(data_path)

# PET

In [3]:
para = ["P", "Tmin", "Tmax"]

list_files = {}

for p in para:
    
    tmp = pd.DataFrame(
        {
            "file": [x for x in all_list_files if x.startswith(p)],
            "year": [int(x.split("_")[1][0:4]) for x in all_list_files if x.startswith(p)]
        }
        
    )
    
    tmp.sort_values(by = "year", inplace = True)
    
    list_files[p] = tmp



In [4]:
ppppp = waterSoilContent(
    covered = 2,
    infiltration= 20,
    evaporation= 5,
    init_swc_evaporation_layer = 25,
    init_swc_transition_layer = 25,
    z_evaporation_layer = 100,
    z_transition_layer = 1000,
    fc_evaporation_layer = 30,
    fc_transition_layer = 30,
    pwp_evaporation_layer = 15,
    pwp_transition_layer = 15,
    stress_coefficient = 0.7,
    MAD = 0.3,
    transpiration = 2,
    init_swc_transpiration_layer = 25,
    pwp_transpiration_layer = 15,
    z_transpiration_layer = 1000,
    fc_transpiration_layer = 30
)
ppppp

(30.0, 300.0, 150.0, 10.0, <NA>, 0, 2, 5, 142.0, 0)

In [5]:
# with open("D:\mehdi\DailyData\land_use.pkl", "rb") as f:
#     land_use = pickle.load(f)

# land_use

# land_use["Plant Date"] = np.datetime64("2022") + land_use['Plant Date'].astype('timedelta64[M]') + np.timedelta64(0, 'D')

In [6]:
# df = land_use.to_dataframe()
# df = df.reset_index()
# column_titles = df.columns.tolist()
# lll = df['cultivated'].to_list()

# unique_list = pd.Series(lll).unique()
# unique_list


In [7]:

# math.isnan(np.unique(land_use['Plant Date'].sel(time = "2022-01-01")[:,:])[-1])

In [8]:
# {year}-{int(x):02d}-{1:02d}

def generate_date(x, year):
    
    if math.isnan(x):
        return "nan"
    else:
        return f"{year}-{int(x):02d}-01"



# plant_date_day = np.vectorize(generate_date)(
#     x = land_use['Plant Date'].sel(time = "2022-01-01")[:,:],
#     year = 2022
# )

# plant_date_day = np.where(plant_date_day == "nan", np.nan, plant_date_day)



In [9]:
# np.unique(plant_date_day)

In [10]:
# year = 2022
# julian_day = 61

# date_object = datetime.datetime.strptime(f"{year}{julian_day}", '%Y%j').date()
# yyy = formatted_date = date_object.strftime("%Y-%m-%d")
# yyy

In [12]:
selected_years = [2010, 2011]

for year in selected_years:
    
    if year == selected_years[0]:
        soil_water_content = xr.open_dataset("D:\mehdi\DailyData\soil_water_content_1979001.nc")
        
        soil_water_content = soil_water_content.fillna(0)
        
        soil_water_content = soil_water_content.where(soil_water_content >= 0, 0)
        
        soil_water_content = soil_water_content.rename({'x': 'lon','y': 'lat'})
        
        with open("D:\mehdi\DailyData\land_use.pkl", "rb") as f:
            land_use = pickle.load(f)
        
        land_use = land_use.rename({'x': 'lon','y': 'lat'})
        
        
        constant_value_for_AE = 6
        AE = xr.Dataset(
            data_vars={"available_evaporable_water": (("time", "lat", "lon"), np.full((len(soil_water_content.time), len(soil_water_content.lat), len(soil_water_content.lon)), constant_value_for_AE))},
            coords={"lat": soil_water_content.lat, "lon": soil_water_content.lon, "time": soil_water_content.time},
        )
    else:
        year_tmp = year - 1
        pass
    
    wp = xr.open_dataset("D:\mehdi\DailyData\wilting_point_1979001.nc")
    wp = wp.rename({'x': 'lon','y': 'lat'})
    
    fc = xr.open_dataset("D:\mehdi\DailyData\coefficient_field_capacity_1979001.nc")
    fc = fc.rename({'x': 'lon','y': 'lat'})
    
    cn = xr.open_dataset("D:\mehdi\DailyData\curve_number_1979001.nc")
    
    cn = cn.fillna(0)
    cn = cn.rename({'x': 'lon','y': 'lat'})
    
    Pre = xr.open_dataset(f"{data_path}{list_files['P'][list_files['P']['year'] == year]['file'].values[0]}")
    
    Tmin = xr.open_dataset(f"{data_path}{list_files['Tmin'][list_files['Tmin']['year'] == year]['file'].values[0]}")
    
    Tmax = xr.open_dataset(f"{data_path}{list_files['Tmax'][list_files['Tmax']['year'] == year]['file'].values[0]}")
    
    lat = np.repeat(Tmin.lat, Tmin.lon.size).values.reshape(Tmin.lat.size, Tmin.lon.size)
    
    lon = np.repeat(Tmin.lon, Tmin.lat.size).values.reshape(Tmin.lat.size, Tmin.lon.size)
    
    Ra = Tmin.copy(deep = True)
    
    Ra = Ra.rename({'air_temperature': 'extraterrestrial_radiation'})
    
    ET = Tmin.copy(deep = True)
    
    ET = ET.rename({'air_temperature': 'et_hargreaves_samani'})
    
    TE = Tmin.copy(deep = True)
    
    TE = TE.rename({'air_temperature': 'total_evaporable_water'})
    
    MRF = Tmin.copy(deep = True)
    
    MRF = MRF.rename({'air_temperature': 'moisture_reduction_function'})
    
    f_adjusted = Tmin.copy(deep = True)
    
    f_adjusted = f_adjusted.rename({'air_temperature': 'moisture_reduction_function'})
    
    ET_actual = Tmin.copy(deep = True)
    
    ET_actual = ET_actual.rename({'air_temperature': 'actual_evapotranspiration'})
    
    K_E = Tmin.copy(deep = True)
    
    K_E = K_E.rename({'air_temperature': 'available_evaporable_water'})
    
    E_actual = Tmin.copy(deep = True)
    
    E_actual = E_actual.rename({'air_temperature': 'actual_evaporation'})
    
    ET_total_actual = Tmin.copy(deep = True)
    
    ET_total_actual = ET_total_actual.rename({'air_temperature': 'total_actual_evapotranspiration'})
    
    RUN_OFF = Tmin.copy(deep = True)
    
    RUN_OFF = RUN_OFF.rename({'air_temperature': 'runoff'})
    
    INFILTRATION = Tmin.copy(deep = True)
    
    INFILTRATION = INFILTRATION.rename({'air_temperature': 'infiltration'})
    
    CROP_COEFFICIENT = Tmin.copy(deep = True)
    
    CROP_COEFFICIENT = CROP_COEFFICIENT.rename({'air_temperature': 'crop_coefficient'})
    
    
    for j_day in range(1, Tmin.time.size + 1):
        
        if j_day != 1:
            pass
        
        print(year, j_day)
        
        date_object = datetime.datetime.strptime(f"{year}{j_day}", '%Y%j').date()
        model_date = date_object.strftime("%Y-%m-%d")
        
        pre_day = Pre.to_array()[0, j_day - 1, :, :]
        
        Tmin_day = Tmin.to_array()[0, j_day - 1, :, :]
        
        Tmax_day = Tmax.to_array()[0, j_day - 1, :, :]
        
        Tmean_day = (Tmin_day + Tmax_day) / 2
        
        ra_day = np.vectorize(extraterrestrial_radiation)(
            inverse_relative_distance_earth_sun(
                julian_date = j_day
            ),
            sunset_hour_angle(
                latitude=convert_degrees2radians(lat),
                solar_declination = solar_declination(
                    julian_date = j_day
                )
            ),
            convert_degrees2radians(
                degrees = lat
            ),
            solar_declination(
                julian_date = j_day
            )            
        )
        
        Ra["extraterrestrial_radiation"][j_day - 1, :, :] = ra_day
                
        et_day_hargreaves_samani = np.vectorize(ReferenceEvapotranspiration.hargreaves_samani)(
            tmin = Tmin_day,
            tmax = Tmax_day,
            tmean = Tmean_day,
            ra = ra_day
        )
        et_day_hargreaves_samani = np.vectorize(lambda d: 0 if d < 0 else d)(
            d = et_day_hargreaves_samani
        )
        
        ET["et_hargreaves_samani"][j_day - 1, :, :] = et_day_hargreaves_samani
        
        total_evaporable_water_day = np.vectorize(available_water)(
            permanent_wilting_point_wet = wp["b0"].sel(time = "1979-01-01")[:, :],
            field_capacity_wet = fc["b0"].sel(time = "1979-01-01")[:, :],
            soil_depth = land_use["Maximum Crop Height (h) (m)"].sel(time = "2022-01-01")[:, :]*1000
        )
        
        TE["total_evaporable_water"][j_day - 1, :, :] = total_evaporable_water_day
        
        soil_water_content_evaporation = soil_water_content["volumetric_soil_water_layer_1"]
        soil_water_content_transpiration = soil_water_content["volumetric_soil_water_layer_2"] + soil_water_content["volumetric_soil_water_layer_3"]
        soil_water_content_transition = soil_water_content["volumetric_soil_water_layer_4"]
        
        f_day = np.vectorize(moisture_reduction_function)(
            soil_wetness_in_previous_step = soil_water_content_transpiration.sel(time = "1979-01-01")[:, :],
            permanent_wilting_point_wet = wp["b0"].sel(time = "1979-01-01")[:, :],
            field_capacity_wet = fc["b0"].sel(time = "1979-01-01")[:, :],
            soil_depth = land_use["Maximum Crop Height (h) (m)"].sel(time = "2022-01-01")[:, :]*1000
        )
        MRF["moisture_reduction_function"][j_day - 1, :, :] = f_day
        
        f_adjusted_day = np.vectorize(lambda d: 0 if d < 0 else d)(
            d = f_day
        )
        
        f_adjusted["moisture_reduction_function"][j_day - 1, :, :] = f_adjusted_day
        
        plant_date_day = np.vectorize(generate_date)(
            x = land_use['Plant Date'].sel(time = "2022-01-01")[:,:],
            year = year
        )
        plant_date_day = np.where(plant_date_day == "nan", pd.NA, plant_date_day)
        
        
        
        crop_coefficient_day = np.vectorize(PotentialEvapotranspiration.calculate_single_crop_coefficient_for_linear_changes_steps)(
            crop_coefficient_ini = land_use["Kc init"].sel(time = "2022-01-01")[:,:],
            crop_coefficient_mid = land_use["Kc mid"].sel(time = "2022-01-01")[:,:],
            crop_coefficient_end = land_use["Kc end"].sel(time = "2022-01-01")[:,:],
            length_ini_crop = land_use["Init. (Lini)"].sel(time = "2022-01-01")[:,:],
            length_dev_crop = land_use["Dev. (Ldev)"].sel(time = "2022-01-01")[:,:],
            length_mid_crop = land_use["Mid (Lmid)"].sel(time = "2022-01-01")[:,:],
            length_late_crop = land_use["Late (Llate)"].sel(time = "2022-01-01")[:,:],
            plant_date = plant_date_day[:,:],
            modeling_date = model_date
        )
        
        
        CROP_COEFFICIENT["crop_coefficient"][j_day - 1, :, :] = crop_coefficient_day
        
        
        et_actual_day = np.vectorize(ActualEvapotranspiration.et_covered)(
            moisture_reduction_function = f_adjusted_day,
            crop_coefficient = CROP_COEFFICIENT["crop_coefficient"].sel(time = model_date)[:,:],
            crop_cover = 0.4,
            reference_crop_evapotranspiration = et_day_hargreaves_samani
        )
        ET_actual["actual_evapotranspiration"][j_day - 1, :, :] = et_actual_day
        
        
        available_evaporable_water_day = AE.to_array()[0, j_day - 1, :, :]
        
        k_e_day = np.vectorize(ratio_of_actual_evaporable_water_to_total_evaporable_water)(
            available_water = total_evaporable_water_day,
            available_evaporable_water = available_evaporable_water_day
        )
        
        K_E["available_evaporable_water"][j_day - 1, :, :] = k_e_day
        
        
        e_actual_day = np.vectorize(ActualEvapotranspiration.e_noncovered)(
            ratio_of_actual_evaporable_water_to_total_evaporable_water = k_e_day,
            crop_cover = 0.4,
            reference_crop_evapotranspiration = et_day_hargreaves_samani
        )
        
        E_actual["actual_evaporation"][j_day - 1, :, :] = e_actual_day
        
        et_total_actual_day = np.vectorize(ActualEvapotranspiration.et_QDWB)(
            evaporation_noncovered_areas = e_actual_day,
            evapotranspiration_covered_areas = et_actual_day
        )
        
        ET_total_actual["total_actual_evapotranspiration"][j_day - 1, :, :] = et_total_actual_day
        
        # ### run off
        
        run_off_and_infiltration_day = np.vectorize(PrimarySurfaceFlow.scs)(
            precipitation = pre_day,
            curve_number = cn["dry"].sel(time = "1979-01-01")[:, :],
            rsa = True,
            antecedent_precipitation = soil_water_content_evaporation.sel(time = "1979-01-01")[:, :],
            is_growing_season = True
        )
        RUN_OFF["runoff"][j_day - 1, :, :] = run_off_and_infiltration_day[0]
        
        INFILTRATION["infiltration"][j_day - 1, :, :] = run_off_and_infiltration_day[1]
        
        ### soil water content
        
        
        soil_water_content_day = waterSoilContent(
            covered = land_use["cultivated"].sel(time = "2022-01-01")[:,:],
            infiltration= run_off_and_infiltration_day[1],
            evaporation= e_actual_day,
            init_swc_evaporation_layer = soil_water_content_evaporation.sel(time = "1979-01-01")[:, :],
            init_swc_transition_layer = soil_water_content_transition.sel(time = "1979-01-01")[:, :],
            z_evaporation_layer = 100,
            z_transition_layer = 1000,
            fc_evaporation_layer = fc["b10"].sel(time = "1979-01-01")[:, :],
            fc_transition_layer = fc["b200"].sel(time = "1979-01-01")[:, :],
            pwp_evaporation_layer = wp["b10"].sel(time = "1979-01-01")[:, :],
            pwp_transition_layer = wp["b200"].sel(time = "1979-01-01")[:, :],
            stress_coefficient = 0.7,
            MAD = 0.3,
            transpiration = et_actual_day,
            init_swc_transpiration_layer = soil_water_content_transpiration.sel(time = "1979-01-01")[:, :],
            pwp_transpiration_layer = wp["b100"].sel(time = "1979-01-01")[:, :],
            z_transpiration_layer = land_use["Maximum Crop Height (h) (m)"].sel(time = "2022-01-01")[:, :]*1000,
            fc_transpiration_layer = fc["b100"].sel(time = "1979-01-01")[:, :]
        )
        
        
        
              
            
        
        # swc_evaporation_layer_day = np.vectorize(SoilContent.evaporation_layer)(
        #     soil_water_content_of_evaporation_layer_at_previous_step = soil_water_content_evaporation,
        #     infiltration = run_off_and_infiltration_day[1],
        #     evaporation = e_actual_day,
        #     field_capacity_soil_water_content_of_evaporation_layer = fc["b10"].sel(time = "1979-01-01")[:, :],
        #     permanent_wilting_point_soil_water_content_of_evaporation_layer = wp["b10"].sel(time = "1979-01-01")[:, :],
        #     coverd = True,
        #     infiltration_to_transpiration_layer = float,
        #     infiltration_to_transition_layer = float
        # )
        
        
        break
    # Ra.to_netcdf(f"{data_path}Ra_{year}.nc")
    
    # ET.to_netcdf(f"{data_path}ET_{year}.nc")
        
    
    break


2010 1


c:\Users\psh2610\AppData\Local\miniconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
np.unique(RUN_OFF["runoff"].sel(time = "2010-01-01")[:,:])

In [ ]:
pd.isnull(np.nan)

In [ ]:
model_date

In [ ]:
np.unique(plant_date_day)

In [ ]:
rt = ET_total_actual["total_actual_evapotranspiration"].sel(time = "2010-01-01")[:, :].plot()
# num_elements_equal_to_10 = (rt < 0).sum()
# num_elements_equal_to_10

In [ ]:
ee = ET_total_actual["total_actual_evapotranspiration"].sel(time = "2010-01-01")
dd = ee.max()
float_value = dd.item()
float_value